**Application of Geometric Brownian Motion (GBM) to calculate European Option Price using Monte Carlo Simulation**

This code estimates the price of a European call option using Monte Carlo simulation by generating multiple paths for the stock price using the GBM model and then calculating the average payoff discounted to present value.

In [1]:
import numpy as np

# Parameters
S0 = 100       # Initial stock price
K = 105        # Strike price
T = 1.0        # Time to maturity in years
r = 0.05       # Risk-free rate
sigma = 0.2    # Volatility
N = 365        # Number of time steps
M = 10000      # Number of simulated paths
dt = T / N     # Time step

np.random.seed(0)  # For reproducibility

# Simulate M stock price paths
paths = np.zeros((M, N+1))
paths[:, 0] = S0
for t in range(1, N + 1):
    Z = np.random.standard_normal(M)  # Random shocks
    paths[:, t] = paths[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

# Calculate the payoff for a European call option
payoffs = np.maximum(paths[:, -1] - K, 0)
option_price = np.exp(-r * T) * np.mean(payoffs)

print(f"Estimated European Call Option Price: {option_price:.2f}")


Estimated European Call Option Price: 7.95


**Next Steps - Applying Polygon.io Stock Options API Data to Price American Options**

The next steps involved in the process is to change the code by applying the Rest API to pull data from Polgon.io's option data which can provide the necessary information to apply the GBM to American options. We will first need to change the code to allow for the options to be exercised at any time which is the difference between American and European options. This feature makes American options more complex to price using Monte Carlo simulation, as you must consider the option's value at every point in time along each simulated path.

A common approach to price American options with Monte Carlo simulation is through the use of the Least Squares Monte Carlo (LSM) method, introduced by Longstaff and Schwartz. The LSM method involves simulating multiple paths for the underlying asset's price, as before, but then uses a regression technique at each time step to estimate the continuation value of holding the option versus the immediate exercise value, deciding the optimal strategy at each step.

1. Simulate the stock price paths as before.
2. At each time step and for each path, calculate the payoff if the option were exercised at that time (for a call, this would be max(S_t -     K, 0), where S_t is the stock price at time t).
3. Use backward induction: starting from the second-to-last time step, estimate the continuation value of the option (if not exercised)         using a regression on the simulated stock prices, and compare it to the immediate exercise value. The option's value at this time and        path is the maximum of the exercise value and the estimated continuation value.
4. Move backward through time, repeating step 3, until you reach the first time step.
5. The option price is the average of the discounted values at the first time step.

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Parameters
S0 = 36.0       # Initial stock price
K = 40.0        # Strike price
T = 1.0         # Time to maturity in years
r = 0.06        # Risk-free rate
sigma = 0.2     # Volatility
div = 0.06      # Dividend yield
N = 50          # Number of time steps
M = 10000       # Number of simulated paths
dt = T / N      # Time step

np.random.seed(123)  # For reproducibility

# Simulate M stock price paths with adjustments for dividends
paths = np.zeros((M, N+1))
paths[:, 0] = S0
for t in range(1, N + 1):
    Z = np.random.standard_normal(M)  # Random shocks
    paths[:, t] = paths[:, t-1] * np.exp((r - div - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

# Function to calculate the payoff for an American option (put in this case)
def american_option_payoff(paths, K):
    return np.maximum(K - paths, 0)

# Initialize the payoff matrix for an American put option
payoffs = american_option_payoff(paths, K)

# Initialize the value matrix (same as payoffs for the last step)
values = np.zeros_like(payoffs)
values[:, -1] = payoffs[:, -1]

# LSM algorithm for American options
for t in range(N-1, 0, -1):
    in_the_money = paths[:, t] < K  # Condition for being in the money for a put option
    X = paths[in_the_money, t]
    Y = values[in_the_money, t+1] * np.exp(-r * dt)
    if len(X) > 0:  # Only proceed if there are paths in the money
        regression = np.polyfit(X, Y, deg=2)
        continuation_values = np.polyval(regression, paths[in_the_money, t])
        exercise_values = payoffs[in_the_money, t]
        # Decide between exercising and continuing
        values[in_the_money, t] = np.where(exercise_values > continuation_values, exercise_values, Y)

# Calculate the option price as the average of the discounted first-step values
american_option_price = np.mean(values[:, 1] * np.exp(-r * dt))

print(f"Estimated American Put Option Price: {american_option_price:.2f}")

Estimated American Put Option Price: 4.18


Here's an example of Python code that allows a user to enter a specific options symbol and date to receive data from the Tradier API.

In [ ]:
import requests

def get_option_chain(symbol, expiration, include_greeks=True):
    API_KEY = 'YOUR_API_KEY_HERE'  # Replace with your Tradier API token
    endpoint = "https://api.tradier.com/v1/markets/options/chains"
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Accept': 'application/json'
    }
    params = {
        'symbol': symbol,
        'expiration': expiration,
        'greeks': str(include_greeks).lower()
    }
    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
symbol = input("Enter the underlying symbol (e.g., AAPL): ")
expiration = input("Enter the expiration date (YYYY-MM-DD): ")
option_data = get_option_chain(symbol, expiration)
print(option_data)


Here's how to integrate the API implementation directly into the American option pricing code. This script will fetch the options chain from Tradier, extract the Greeks data, and use it to calculate the price of an American option using the Binomial model: